In [ ]:
def stage1Meeting(stage1):
    #终止计算条件-到达发射点了
    #同时过滤已经到发射点的车
    should,st=[],{}
    for car,state in stage1.items():
        a=state["D-F时序"]["位置"].iloc[state["当前位置"]+1]
        b=state["发射点"]
        if a!=b:
            should.append(car) 
    if len(should)==0:
        return stage1
    else:
    #装填过滤的车
        for i in should:
            st[i]=stage1[i]
    
    #耗时计算
    t=[(car,state["出发时刻"]+sum(state["D-F时序"]['耗时'].iloc[1:state["当前位置"]+1])) for car,state in st.items()]
    tdic={car:state["出发时刻"]+sum(state["D-F时序"]['耗时'].iloc[1:state["当前位置"]+1]) for car,state in st.items()}
    
    #t=sorted(t, key=lambda item: item[1])
    #fastCar,fastTime=t[0][0],t[0][1]
    tz=min(t, key=lambda item: item[1])
    fastCar,fastTime=tz[0],tz[1]
    
    #时间线比较
    tt=st[fastCar]["D-F时序"].iloc[st[fastCar]['当前位置']+2]
    if tt['类型'] != 'edge':
        print("出错了：001")
        return stage1
    ttt=tt["位置"]
    roadSize=G.edge[ttt[0]][ttt[1]]['容量']
    #时间线比较-之重叠
    meetingCars=[]
    for car,state in st.items():
        loc=state["D-F时序"]['位置'].iloc[state["当前位置"]]
        if set(ttt)==set(loc):
            meetingCars.append(car)
    
    #计算其是否在区间时间范围内
    amtz=[]
    for car in meetingCars:
        a=sum(stage1[car]["D-F时序"]['耗时'].iloc[1:stage1[car]["当前位置"]+1])
        b=sum(stage1[car]["D-F时序"]['耗时'].iloc[1:stage1[car]["当前位置"]])

        if a <= fastTime and fastTime < b:
            amtz.append(car)
    meetingCars=amtz
    
    if len(meetingCars)>=roadSize:
        #说明需要会车等待
        if len(meetingCars)>roadSize:
            print(meetingCars)
            print("出错了：002")
            return stage1
        #To-do 会车等待决策+1
        waitTime = min([tdic[i] for i in meetingCars]) - fastTime
        #写入等待时间
        stage1[fastCar]["D-F时序"].loc[stage1[fastCar]['当前位置']+1,'耗时']=waitTime
    stage1[fastCar]['当前位置']=stage1[fastCar]['当前位置']+2
    
    return stage1Meeting(stage1)    

In [ ]:
def stage2Meeting(stage2):
    #终止条件-齐射
    should,st=[],{}
    for car,state in stage2.items():
        a=state["Z-F时序"]["位置"].iloc[state["当前Z-F位置"]+1]
        b=state["发射点"]
        if a!=b:
            should.append(car) 
    if len(should)==0:
        return stage2
    
    #过滤到达终点的
    #装填过滤的车
    for i in should:
        st[i]=stage2[i]
    
    #重叠检测
    #耗时计算
    t,tdic=[],{}
    for car,state in st.items():
        if state["转载完成"]:
            h=sum(state["Z-F时序"]['耗时'].iloc[1:state["当前Z-F位置"]+1])+sum(state["F-Z时序"]['耗时'].iloc[1:state["当前F-Z位置"]+1])
        else:
            h=sum(state["F-Z时序"]['耗时'].iloc[1:state["当前F-Z位置"]+1])
        t.append((car,h))
        tdic[car]=h
            
    #t=sorted(t, key=lambda item: item[1])
    #fastCar,fastTime=t[0][0],t[0][1]
    tz=min(t, key=lambda item: item[1])
    fastCar,fastTime=tz[0],tz[1]
    
    #时间线比较
    if st[fastCar]["转载完成"]:
        tt=st[fastCar]["Z-F时序"].iloc[st[fastCar]['当前Z-F位置']+2]
        if tt['类型'] != 'edge':
            print("出错了：003")
            return stage2
        ttt=tt["位置"]
        roadSize=G.edge[ttt[0]][ttt[1]]['容量']
        #时间线比较-之重叠
        meetingCars=[]
        for car,state in st.items():
            if state["转载完成"]:
                loc=state["Z-F时序"]['位置'].iloc[state["当前Z-F位置"]]
            else:
                loc=state["F-Z时序"]['位置'].iloc[state["当前F-Z位置"]]
            if set(ttt)==set(loc):
                meetingCars.append(car)
        
        #计算其是否在区间时间范围内
        amtz=[]
        for car in meetingCars:
            if state["转载完成"]:
                a=sum(stage2[car]["Z-F时序"]['耗时'].iloc[1:stage2[car]["当前Z-F位置"]+1])+sum(stage2[car]["F-Z时序"]['耗时'].iloc[1:stage2[car]["当前F-Z位置"]+1])
                b=sum(stage2[car]["Z-F时序"]['耗时'].iloc[1:stage2[car]["当前Z-F位置"]])+sum(stage2[car]["F-Z时序"]['耗时'].iloc[1:stage2[car]["当前F-Z位置"]+1])
            else:
                a=sum(stage2[car]["F-Z时序"]['耗时'].iloc[1:stage2[car]["当前F-Z位置"]+1])
                b=sum(stage2[car]["F-Z时序"]['耗时'].iloc[1:stage2[car]["当前F-Z位置"]])

            if a <= fastTime and fastTime < b:
                amtz.append(car)
        meetingCars=amtz
        
        if len(meetingCars)>=roadSize:
            #说明需要会车等待
            if len(meetingCars)>roadSize:
                print("出错了：004")
                return stage2
            #To-do 会车等待决策+1
            waitTime = min([tdic[i] for i in meetingCars]) - fastTime
            #写入等待时间
            stage2[fastCar]["Z-F时序"].loc[stage2[fastCar]['当前Z-F位置']+1,'耗时']=waitTime
        stage2[fastCar]['当前Z-F位置']=stage2[fastCar]['当前Z-F位置']+2
    else:
        tt=st[fastCar]["F-Z时序"].iloc[st[fastCar]['当前F-Z位置']+2]
        if tt['类型'] != 'edge':
            print("出错了：005")
            return stage2
        ttt=tt["位置"]
        roadSize=G.edge[ttt[0]][ttt[1]]['容量']
        #时间线比较-之重叠
        meetingCars=[]
        for car,state in st.items():
            if state["转载完成"]:
                loc=state["Z-F时序"]['位置'].iloc[state["当前Z-F位置"]]
            else:
                loc=state["F-Z时序"]['位置'].iloc[state["当前F-Z位置"]]
            if set(ttt)==set(loc):
                meetingCars.append(car)
        #计算其是否在区间时间范围内
        amtz=[]
        for car in meetingCars:
            if state["转载完成"]:
                a=sum(stage2[car]["Z-F时序"]['耗时'].iloc[1:stage2[car]["当前Z-F位置"]+1])+sum(stage2[car]["F-Z时序"]['耗时'].iloc[1:stage2[car]["当前F-Z位置"]+1])
                b=sum(stage2[car]["Z-F时序"]['耗时'].iloc[1:stage2[car]["当前Z-F位置"]])+sum(stage2[car]["F-Z时序"]['耗时'].iloc[1:stage2[car]["当前F-Z位置"]+1])
            else:
                a=sum(stage2[car]["F-Z时序"]['耗时'].iloc[1:stage2[car]["当前F-Z位置"]+1])
                b=sum(stage2[car]["F-Z时序"]['耗时'].iloc[1:stage2[car]["当前F-Z位置"]])

            if a <= fastTime and fastTime < b:
                amtz.append(car)
        meetingCars=amtz
        if len(meetingCars)>=roadSize:
            #说明需要会车等待
            if len(meetingCars)>roadSize:
                print(meetingCars)
                print("出错了：006")
                return stage2
            #To-do 会车等待决策+1
            waitTime = min([tdic[i] for i in meetingCars]) - fastTime
            #写入等待时间
            stage2[fastCar]["F-Z时序"].loc[stage2[fastCar]['当前F-Z位置']+1,'耗时']=waitTime
        #时间线比较-之转载爆仓
        zt=st[fastCar]["F-Z时序"].iloc[st[fastCar]['当前F-Z位置']+3]
        if zt['类型'] != 'node':
            print("出错了：007")
            return stage2
        if zt['位置'] == st[fastCar]['转载点']:
            #检测是否爆仓
            #所有是这个转载点的车
            #且到过转载点的
            stz=[car for car,state in stage2.items() if state["转载完成"] and state['转载点']==zt['位置']]
            ztTime=sum(st[fastCar]["F-Z时序"]['耗时'].iloc[1:st[fastCar]['当前F-Z位置']+2])
            #计算其是否在区间时间范围内
            stzz,mtz=[],[]
            for car in stz:
                a=sum(stage2[car]["F-Z时序"]['耗时'].iloc[1:-1])
                b=sum(stage2[car]["F-Z时序"]['耗时'].iloc[1:])
                if a <= ztTime and ztTime < b:
                    mtz.append(b)
                    stzz.append(car)
            #基础转载时间
            zz=10
            if len(stzz) >= 2:
                if len(stzz) > 2:
                    print("报错：特殊")
                #说明爆仓
                waitt=min(mtz)-ztTime
                stage2[fastCar]["F-Z时序"].loc[stage2[fastCar]['当前F-Z位置']+1,'耗时']=waitt
                #计算新的到达时间
                ztt=sum(st[fastCar]["F-Z时序"]['耗时'].iloc[1:st[fastCar]['当前F-Z位置']+2])
                zz+=max(mtz)-ztt
            #未爆仓，进场计算转载耗时
            if len(stzz) ==1:
                #表示存在正在转载物
                zz+=sum(stage2[stzz[0]]["F-Z时序"]['耗时'].iloc[1:])-ztTime
            #并写入转载完成和位置，转载时间
            stage2[fastCar]['转载完成']=True
            stage2[fastCar]['当前F-Z位置']=stage2[fastCar]['当前F-Z位置']+3
            stage2[fastCar]["F-Z时序"].loc[stage2[fastCar]['当前F-Z位置'],'耗时']=zz
        stage2[fastCar]['当前F-Z位置']=stage2[fastCar]['当前F-Z位置']+2

    return stage2Meeting(stage2)
    